# Cloze Filling-基于BiLSTM的完形填空问题

In [1]:
print("""-----董政宇 2023年4月23日12:36:05-----""")

-----董政宇 2023年4月23日12:36:05-----


In [ ]:
'''
开始: 数据下载与导入
'''
# eg：
# 输入：‘通过贺星龙孩子讲述的故事，我曾看到了一位?毅的父亲，一位全心全意为村民服务的乡村’
# 输出：‘坚’
# 简单来说，就是把？预测出来
# from datasets import load_dataset
# dataset = load_dataset("seamew/ChnSentiCorp")

In [1]:
'''
step1: 数据预处理
'''
import os
import torch 
import torch.nn as nn
import torch.utils.data as Data
import random
import torch.utils.data as Data

# 超参数设置
BATCH_SIZE = 32
LR = 1e-4
NUM_HIDDENS = 256
SEQ_LEN = 40
WORD_DIM = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
path = '../dataTry/biLSTM_data'
# chinese_data_path = os.path.join(path, 'chinese-data.txt')
chinese_data_path = os.path.join(path, 'train.csv')
origin_data,predict_word = [],[]
# vocab
vocab = []
# str to index
s2i = dict()
# index to str
i2s = dict()
index = 0

def get_vocab(path):
    chinese_data_path = path
    with open(chinese_data_path,'r', encoding='utf-8') as f:
        # print('loading txt...')
        for line in f:
            if len(line) >= 41:
            	# 需要对？进行特殊处理
                # print(line)
                line = line.replace("?", "。")
                line = line.replace("？", "。")
                predict_word.append(line[20])
                d = line[:20]+'?'+line[21:40]
                origin_data.append(d)
                for word in line:
                    if word not in vocab:
                        vocab.append(word)
    f.close()
get_vocab(chinese_data_path)

# vocab size
vocab.append('?')
vocab_size = len(vocab)

random.shuffle(vocab)
s2i = {
            word: i for i, word in enumerate(vocab)}
i2s = {
            i :word for i, word in enumerate(vocab)}

In [16]:
vocab_size, s2i['?'], i2s[264], len(predict_word), origin_data[11]

(4241, 264, '?', 8458, '1,"入住的是度假区的豪华海景房,前台给?5楼(最高6楼),然后差不多100%的')

In [20]:
origin_data[8000:8005], predict_word[8000:8005]

(['1,看了《饮食定生死》，觉得它是一本通俗?懂的健康科普读物。从中我认识到了食物对',
  '0,USB接口只有两个，少了点~ 13英?的应该再轻薄一点就好了~ 操作系统vi',
  '1,用的还可以，可惜不能帮忙装内存条。而?我同学的第二个盘双击就可以备份，我的不',
  '0,第一次订到这么差的房间，可能是黄金周?关系，价格和档次根本不等同，纱窗是坏的',
  '1,刚买了不到一个月，前两天觉得用起来非?累眼睛无意间发现原来屏幕跳的特别厉害（'],
 ['易', '寸', '且', '的', '常'])

In [23]:
'''
step2: 数据集分割 数据迭代器构造
'''
class Dataset(Data.Dataset):
    def __init__(self,data):
        super(Dataset, self).__init__()
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x,y = self.data[index]
        # print(x,y)
        x = [s2i[i] for i in x]
        y = s2i[y]
        return torch.LongTensor(x),torch.tensor(y)

data = list(zip(origin_data,predict_word))
# 随机打乱data_list顺序
random.shuffle(data)
# 前8000条为训练数据集
train_dataset = Dataset(data[:8000])
# 8000条后为测试数据集
test_dataset = Dataset(data[8000:])
# 训练数据集迭代器
train_loader = Data.DataLoader(
    dataset=train_dataset,
    shuffle=True,
    # 董政宇 改 2023年4月23日20:37:46
    num_workers=0,#原num_workers=0，多线程？
    batch_size=BATCH_SIZE,
    drop_last=True
)
# 测试数据集迭代器
test_loader = Data.DataLoader(
    dataset=test_dataset,
    shuffle=False,
    # 董政宇 改 2023年4月23日20:37:46
    num_workers=0,#原num_workers=0，多线程？
    batch_size=BATCH_SIZE,
    drop_last=True
)

len(test_dataset),len(train_dataset)

(458, 8000)

In [37]:
print(len(data))
# data[:10]
torch.tensor(264)
x="1,看了《饮食定生死》，觉得它是一本通俗?懂的健康科普读物。从中我认识到了食物对"
print(len(x))
x = [s2i[i] for i in x]
torch.LongTensor(x), len(torch.LongTensor(x))

8458
40


(tensor([  66, 2763,  940,  874, 1544,  177, 2530,   36,  814, 1164,  614, 3560,
         1232, 2928,  320, 3454, 4141, 1645, 1754,   14,  264, 2652, 1695, 3124,
         2440, 3979, 1888, 2782, 2774,  782, 4075, 2518, 2718, 1006,  483, 1925,
          874, 2530, 2774, 3146]),
 40)

In [5]:
'''
step3: BiLSTM双向长短期记忆网络定义
'''
EPOCH = 10
import torch.nn.functional as F
class BiLSTM(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=WORD_DIM,
            hidden_size=NUM_HIDDENS,
            # 开启双向BiLSTM
            bidirectional=True
          )
        self.out = nn.Linear(NUM_HIDDENS*2,vocab_size)
    def forward(self,x):
        # print(x.shape)
        # x.shape [batch,seq_len,word_dim]     torch.Size([32, 40, 128])
        # state.shape [1,seq_len,word_dim]
        h_s = torch.randn((1*2,BATCH_SIZE, NUM_HIDDENS),device=device)
        c_s = torch.randn((1*2,BATCH_SIZE, NUM_HIDDENS),device=device)
        output,(_,_)= self.lstm(x.transpose(1,0),(h_s,c_s)) # output.shape [seq_len,batch,num_hiddens] torch.Size([40, 32, 256])
        chinese = self.out(output[20])  # chinese.shape [batch,vocab_size]
       
        return chinese 
       
def encode(data,embed):
    return embed(data)

def train_porcess(pred,acc,device):
    # pred.shape [batch,vocab_size]
    # acc.shape [batch]
    # print(pred.shape)torch.Size([32, 3195])

    pred = F.softmax(pred,dim=-1)
    # print(pred.shape)torch.Size([3195])
    pred = pred.argmax(dim=-1).cpu().numpy()
    acc = acc.cpu().numpy()
    print('预测下一个字:',[i2s[i] for i in pred.tolist()])
    print('实际下一个字:',[i2s[i] for i in acc.tolist()])

In [6]:
'''
step4: 实例化和初始化
'''
bilstm = BiLSTM(vocab_size)
criterion = nn.CrossEntropyLoss()
optim = torch.optim.Adam(bilstm.parameters(),lr=LR)
embed = nn.Embedding(vocab_size,WORD_DIM,device = device)
output_model = 'models/BiLSTM_models/chineseBiLSTM_model.pth'
COUNT = 0

In [9]:
'''
注意: 初始化模型参数（从头开始训练）
'''
torch.save(
            {
            
                'model_state_dict': bilstm.state_dict(),
                'optimizer_state_dict': optim.state_dict()
            }, output_model
          )

In [13]:
'''
step5: 开始训练(继续训练)
'''
BATCH_SIZE =32
if __name__ == '__main__':
    
    '''这段注释掉的代码意义何在 研究ing'''
    # 加载模型（接上次训练参数继续训练？）
    checkpoint = torch.load(output_model,map_location='cpu')
    # print(checkpoint)
    bilstm.load_state_dict(checkpoint['model_state_dict'],strict=False)
    bilstm.to(device)
    bilstm.train()
    bestscore = 0
    for epoch in range(10):
        # print(f'epoch = {epoch}')
        print(f'--------epoch = {COUNT*10 + epoch + 1}--------') 
        accuracy =0
        time=0
        # 训练数据集开始训练并计算loss损失  
        for step,datas in enumerate(train_loader):
            data,label = tuple(t.to(device)for t in datas)
            data = encode(data,embed)
            chinese = bilstm(data)
            loss = criterion(chinese,label.long())
            optim.zero_grad()
            loss.backward()
            optim.step()
            if step%10 == 0:
                print(f'loss = {loss}')
                
            if step%50 == 0:
                train_porcess(chinese,label,device)
        bilstm.eval()  
        # 测试数据集开始评估并计算accuracy  
        for step,datas in enumerate(test_loader):
            # print(datas)
            data,label = tuple(t.to(device)for t in datas)
            data = encode(data,embed)
            chinese = bilstm(data)
            
            pred = F.softmax(chinese,dim=-1)       
            pred = pred.argmax(dim=-1).cpu().numpy()
            acc = label.cpu().numpy()
            accuracy += sum(acc==pred)
            time += len(acc)
        # 测试数据集的精确度
        accuracy/=time+0.0
        print('测试数据集的accuracy为：\n',accuracy)
        #保存最优模型的参数（得分最高的模型参数）
        if accuracy>bestscore:
            torch.save(
            {
            
                'model_state_dict': bilstm.state_dict(),
                'optimizer_state_dict': optim.state_dict()
            }, output_model
          )
            bestscore = accuracy
        bilstm.train()
    COUNT = COUNT + 1

--------epoch = 21--------
loss = 1.147019863128662
预测下一个字: ['，', '书', '舒', '，', '，', '烂', '的', '了', '太', 'O', '干', '的', '一', '不', '流', '，', 'Z', '看', '，', '认', '何', '2', '，', '往', '是', '。', '，', '还', '前', '装', '。', '了']
实际下一个字: ['，', '书', '舒', '，', '，', '烂', '人', '了', '太', 'O', '干', '人', '一', '不', '流', '，', 'Z', '看', '，', '认', '何', '2', '，', '往', '本', '收', '，', '还', '前', '装', '。', '本']
loss = 1.5085718631744385
loss = 1.2544234991073608
loss = 1.185636281967163
loss = 1.4971365928649902
loss = 1.4491740465164185
预测下一个字: ['楼', '容', '孩', '样', '未', '调', '用', '机', '的', '八', '2', '历', '上', 'O', '能', '的', '差', '占', '是', '感', '一', '备', '么', '便', '，', '意', '间', '般', '有', '尤', '去', '灼']
实际下一个字: ['楼', '容', '孩', '样', '未', '调', '用', '机', '的', '八', '2', '历', '上', 'O', '能', '的', '差', '占', '笔', '次', '开', '备', '么', '便', '，', '意', '正', '般', '有', '尤', '去', '灼']
loss = 0.9811719655990601
loss = 1.3725101947784424
loss = 1.3823106288909912
loss = 1.1218723058700562
loss = 1.3440744876861572
预测下一个字: ['，',